In [1]:
import pandas as pd
import pennylane as qml
import numpy as np
from pennylane.transforms import  merge_amplitude_embedding

In [88]:
entrada_train= [1,0,1,1,1]
entrada_test=  [0,1,1,1,1]
qubit_number_AmplitudeEmbedding = int(np.ceil(np.log2(len(entrada_train)))) if np.log2(len(entrada_train)) > 0 else 1


In [25]:
def normalize_vector(vector):
    # Calcula la magnitud del vector
    magnitude = np.linalg.norm(vector)
    # Divide cada elemento del vector por la magnitud
    normalized_vector = vector / magnitude
    return normalized_vector

#normalized = normalize_vector(entrada)
#print("Vector normalizado:", normalized)

In [89]:
dev = qml.device("default.qubit", 3*qubit_number_AmplitudeEmbedding)
#dev_qiskit = qml.device("qiskit.aer", qubits_totales)

@qml.qnode(device=dev)
@qml.transforms.merge_amplitude_embedding
def circuit(vector_test,vector_train):
    qml.AmplitudeEmbedding(features=vector_test, wires=range(qubit_number_AmplitudeEmbedding,2*qubit_number_AmplitudeEmbedding), normalize=True, pad_with=0.)
    qml.AmplitudeEmbedding(features=vector_train, wires=range(2*qubit_number_AmplitudeEmbedding,3*qubit_number_AmplitudeEmbedding), normalize=True, pad_with=0.)
    #return qml.probs(wires=range(dev.num_wires))
    estado = qml.state
    ancillea = []
    for i in range(qubit_number_AmplitudeEmbedding):
        qml.Hadamard(i)
        anc = i
        ancillea.append(anc)
        first_state = i+qubit_number_AmplitudeEmbedding
        second_state = i+ 2*qubit_number_AmplitudeEmbedding
        qml.CSWAP(wires=[anc, first_state, second_state])
        qml.Hadamard(i)
    return qml.expval(qml.operation.Tensor(*[qml.PauliZ(i) for i in ancillea]))
    #qml.probs(wires = range(len(ancillea)))

estado_0 = circuit(entrada_test,entrada_train)
print(qml.draw(circuit)(entrada_test,entrada_train))
print("\n")
print(estado_0)

0: ──H──────────────────────╭●─────H───────────────────┤ ╭<Z@Z@Z>
1: ─────────────────────────│──────H─╭●─────H──────────┤ ├<Z@Z@Z>
2: ─────────────────────────│────────│──────H─╭●─────H─┤ ╰<Z@Z@Z>
3: ─╭AmplitudeEmbedding(M0)─├SWAP────│────────│────────┤         
4: ─├AmplitudeEmbedding(M0)─│────────├SWAP────│────────┤         
5: ─├AmplitudeEmbedding(M0)─│────────│────────├SWAP────┤         
6: ─├AmplitudeEmbedding(M0)─╰SWAP────│────────│────────┤         
7: ─├AmplitudeEmbedding(M0)──────────╰SWAP────│────────┤         
8: ─╰AmplitudeEmbedding(M0)───────────────────╰SWAP────┤         
M0 = 
[0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.25+0.j 0.  +0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.25+0.j 0.  +0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.25+0.j 0.  +0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.25+0.j 0.  +0.j 0.25+0.j 0.25+0.j 0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j
 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.

In [108]:
dev2 = qml.device('default.qubit', wires=6)
@qml.qnode(device=dev2)
@qml.transforms.merge_amplitude_embedding
def circuit():
    qml.Hadamard(0)
    qml.Hadamard(1)
    qml.AmplitudeEmbedding([0,1,0,0], wires = [2,3], normalize=True)
    qml.AmplitudeEmbedding([1,1,1,1], wires = [4,5], normalize=True)
    qml.CSWAP(wires=[0,2,4])
    qml.CSWAP(wires=[1,3,5])
    qml.Hadamard(0)
    qml.Hadamard(1)
    return qml.expval(qml.PauliZ(0)@ qml.PauliZ(1))
print(qml.draw(circuit)())
#print(dev2.state)
print(circuit())

0: ──H──────────────────────╭●─────H───────┤ ╭<Z@Z>
1: ──H──────────────────────│─────╭●─────H─┤ ╰<Z@Z>
2: ─╭AmplitudeEmbedding(M0)─├SWAP─│────────┤       
3: ─├AmplitudeEmbedding(M0)─│─────├SWAP────┤       
4: ─├AmplitudeEmbedding(M0)─╰SWAP─│────────┤       
5: ─╰AmplitudeEmbedding(M0)───────╰SWAP────┤       
M0 = 
[0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j 0. +0.j
 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
0.24999999999999978


In [109]:
tensor_product = np.kron([0,1,0,0], [1,1,1,1])
print(tensor_product)

[0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0]
